In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import seaborn as sns
import feature_selection as fs
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
import prep_utils as pu
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pointbiserialr
from sklearn.feature_selection import RFE

PAST_YEARS = 1
TEST_YEAR = 10
KAGGLE_TEST_YEAR = 11

In [2]:
db = sqlite3.connect("db/ac2.db")
db_cur = db.cursor()

[df_awards, df_coaches, df_players_teams, df_players, df_series_post, df_teams_post, df_teams] = pu.db_to_pandas(db)

In [3]:
# Transform all possible attributes into percentages. (Made / Attempted) & (Offensive & Defensive Rebound %)
df_new_teams = pu.prepare_teams(df_teams,df_teams_post,PAST_YEARS)

df_new_teams = fs.fs_teams(df_new_teams)

df_new_teams = pu.playoff_rank(df_new_teams,df_teams,PAST_YEARS)
df_team_results = df_new_teams[["year","tmID","confID","playoff","rank","team_playoffs_count","playoff_rank"]]


Dropping divID in Teams...


In [4]:
df_new_coaches = pu.prepare_coaches(df_coaches, df_awards,PAST_YEARS)
df_new_coaches = pu.group_coaches(df_new_coaches)

df_new_coaches.drop("coachID", axis = 1, inplace = True)

df_final_coaches = df_new_coaches.copy()
df_final_coaches.columns = df_final_coaches.columns.str.lower()


Dropping Attribute lgID in Coaches...
Creating attribute coach previous regular season win ratio...
Creating attribute coach playoffs win ratio...
Creating attribute coach playoffs count...
Creating attribute coach awards count...
Dropping attribute post_wins..
Dropping attribute post_losses..
Dropping attribute won..
Dropping attribute lost..

Coaches Null Verification:
year                    0
tmID                    0
coachID                 0
coach_reg_wr            0
coach_po_wr             0
coach_playoffs_count    0
coach_awards            0
dtype: int64


In [5]:
df_new_players_teams = pu.prepare_player_teams(df_players_teams,df_awards,PAST_YEARS)
# How the team performed in the previous year


previous_team_ratings, df_new_players_team = pu.final_team_ratings(df_players_teams,df_awards, df_players, df_teams, PAST_YEARS)
# Get year 2 sorted by team
"""p = previous_team_ratings[previous_team_ratings['year'] == 2]
p = p.sort_values(by=['tmid'])
print(p.to_string())"""


# How the players performed in the previous year
previous_team_player_ratings = pu.final_player_team_ratings(df_teams, df_new_players_team, df_players, PAST_YEARS,df_new_players_teams[df_new_players_teams['year'] == KAGGLE_TEST_YEAR])


df_players = df_new_players_teams.copy()
df_players = fs.fs_players(df_players,0.2)
df_players = df_players[df_players['year'] != 1]


df_team_results.columns = df_team_results.columns.str.lower()
previous_team_player_ratings.columns = previous_team_player_ratings.columns.str.lower()


merged_data = pd.merge(df_players, df_team_results, on=['tmid', 'year'], how='left')
merged_data = pd.merge(merged_data, df_final_coaches, on=['tmid', 'year'], how='left')
merged_data = pd.merge(merged_data, previous_team_ratings, on=['tmid', 'year'], how='left')
merged_data = pd.merge(merged_data, previous_team_player_ratings, on=['tmid', 'year'], how='left')




Dropping Attribute lgID in Players_Teams...


c:\Users\ricar\Desktop\AC\FEUP-AC\prep_utils.py:741: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data.sort_values(by='year', inplace=True)
c:\Users\ricar\Desktop\AC\FEUP-AC\prep_utils.py:744: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['NextYear_tmID'] = player_data['tmID'].shift(-1)
c:\Users\ricar\Desktop\AC\FEUP-AC\prep_utils.py:745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

     year tmID  Team_Rating  Team_Players_Rating
0     2.0  CHA  1609.898059             0.381618
1     2.0  CLE  1746.666413             0.448017
2     2.0  DET  1309.679589             0.235866
3     2.0  HOU  1184.912468             0.175294
4     2.0  IND  1669.394371             0.410502
5     2.0  LAS  2351.135275             0.741477
6     2.0  MIA  1371.628886             0.265942
7     2.0  MIN  1023.284204             0.096826
8     2.0  NYL  2446.002670             0.787534
9     2.0  ORL  1681.103513             0.416187
10    2.0  PHO  1443.523362             0.300845
11    2.0  POR  1679.110975             0.415219
12    2.0  SAC  2295.525987             0.714479
13    2.0  SEA  1224.010448             0.194275
14    2.0  UTA  1794.939191             0.471452
15    2.0  WAS  1745.195039             0.447302
16    3.0  CHA  2145.184869             0.641491
17    3.0  CLE  1602.678102             0.378112
18    3.0  DET  1223.443831             0.194000
19    3.0  HOU  2289

c:\Users\ricar\Desktop\AC\FEUP-AC\prep_utils.py:741: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data.sort_values(by='year', inplace=True)
c:\Users\ricar\Desktop\AC\FEUP-AC\prep_utils.py:744: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['NextYear_tmID'] = player_data['tmID'].shift(-1)
c:\Users\ricar\Desktop\AC\FEUP-AC\prep_utils.py:745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

#### Point Bisserial Correlation 
We will use this to check correlation between continuous attributes & target


In [6]:
data1_10 = merged_data[merged_data['year'] != 11]
fs.bisserial_corr(data1_10)

team_players_rating: 39.42% correlation
total_assists: 36.39% correlation
playoff_rank: 33.90% correlation
total_gs: 32.71% correlation
total_points: 31.77% correlation
coach_po_wr: 31.26% correlation
total_minutes: 31.16% correlation
coach_reg_wr: 30.47% correlation
total_turnovers: 28.44% correlation
player_awards: 27.79% correlation
total_blocks: 26.30% correlation
team_rating: 25.33% correlation
total_steals: 24.90% correlation
coach_playoffs_count: 24.50% correlation
total_pf: 23.19% correlation
team_playoffs_count: 19.10% correlation
rank: 18.01% correlation
total_drebounds_pct: 12.93% correlation
total_orebounds_pct: 12.93% correlation
coach_awards: 12.93% correlation
total_dq: 12.35% correlation
total_fg_pct: 10.58% correlation
total_gp: 10.20% correlation
total_ft_pct: 4.27% correlation
total_three_pct: 3.71% correlation


### Dividing the dataset in both train & test
We will be using year 10 and test and the remaining ones to train the model

In [7]:
label_encoder = LabelEncoder()
print(merged_data[merged_data['year'] == 11].to_string())
merged_data['tmid'] = label_encoder.fit_transform(merged_data['tmid'])
merged_data['confid'] = label_encoder.fit_transform(merged_data['confid'])

x = merged_data.drop('playoff', axis=1)
y = merged_data['playoff']

x_train = merged_data[merged_data['year'].between(0, TEST_YEAR - 1)].drop('playoff', axis=1)
y_train = merged_data[merged_data['year'].between(0, TEST_YEAR - 1)]['playoff']

x_test = merged_data[merged_data['year'] == TEST_YEAR].drop('playoff', axis=1)
y_test = merged_data[merged_data['year'] == TEST_YEAR]['playoff']

     year tmid  player_awards  total_minutes  total_points  total_assists  total_steals  total_blocks  total_turnovers  total_pf  total_dq  total_gs  total_gp  total_fg_pct  total_ft_pct  total_three_pct  total_orebounds_pct  total_drebounds_pct confid  playoff  rank  team_playoffs_count  playoff_rank  coach_reg_wr  coach_po_wr  coach_playoffs_count  coach_awards  team_rating  team_players_rating
126    11  ATL              3         6267.4        2382.0          507.2         277.0         130.2            450.4     604.4       4.8     149.2     313.8      0.452721      0.726693         0.322600             0.311232             0.688768     EA      NaN   2.0                    1           8.0      0.529412     0.000000                     1             1     0.813081             0.727730
127    11  CHI              1         3225.8        1166.6          208.6         102.4          68.4            250.8     335.4       1.6      70.4     188.8      0.436040      0.724484         0.392

### RFE
We will running RFE on the different models to find out which features produce the best results

In [8]:
print(merged_data.columns)
min_features = 5

rfe_classifiers = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42,max_iter=10000),
    'Support Vector Machine': SVC(random_state=42, kernel='linear'),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
}

classifiers_features = {}

total_features = len(x_train.columns)

best_model_info = {}


for model_name, model in rfe_classifiers.items():
    print(f"\033[1mModel: {model_name}\033[0m")
    
    results = []

    for i in range(min_features, total_features):
        rfe = RFE(model, n_features_to_select=i)
        rfe.fit(x_train, y_train)
        
        selected_features = set(x_train.columns[rfe.support_])
        selected_features.add("tmid")
        selected_features.add("year")
        selected_features = list(selected_features)

        model.fit(x_train[selected_features], y_train)

        accuracy = model.score(x_test[selected_features], y_test)

        results.append((selected_features, accuracy))
        
        
        
        
        
        

    # Sort the results based on accuracy in descending order
    results = sorted(results, key=lambda x: x[1], reverse=True)

    classifiers_features[model_name] = results[0][0]
    best_model_info[model_name] = {'features': results[0][0], 'accuracy': results[0][1], 'model': model}
    
    # Print the results
    for features, accuracy in results[:3]:
        print("Selected Features:", features)
        print("Accuracy:" + str(accuracy) + '\n')

best_model_name = max(best_model_info, key=lambda k: best_model_info[k]['accuracy'])
best_model = best_model_info[best_model_name]['model']
best_features = best_model_info[best_model_name]['features']

print(f"\033[1mBest Model: {best_model_name}\033[0m")
print(f"Best Features: {best_features}")
print(f"Accuracy: {best_model_info[best_model_name]['accuracy']}\n")

Index(['year', 'tmid', 'player_awards', 'total_minutes', 'total_points',
       'total_assists', 'total_steals', 'total_blocks', 'total_turnovers',
       'total_pf', 'total_dq', 'total_gs', 'total_gp', 'total_fg_pct',
       'total_ft_pct', 'total_three_pct', 'total_orebounds_pct',
       'total_drebounds_pct', 'confid', 'playoff', 'rank',
       'team_playoffs_count', 'playoff_rank', 'coach_reg_wr', 'coach_po_wr',
       'coach_playoffs_count', 'coach_awards', 'team_rating',
       'team_players_rating'],
      dtype='object')
Model: Random Forest
Selected Features: ['total_assists', 'total_three_pct', 'total_turnovers', 'total_fg_pct', 'total_orebounds_pct', 'total_points', 'total_steals', 'total_drebounds_pct', 'team_players_rating', 'team_rating', 'year', 'tmid']
Accuracy:0.8461538461538461

Selected Features: ['total_assists', 'coach_reg_wr', 'total_three_pct', 'year', 'total_turnovers', 'total_fg_pct', 'total_orebounds_pct', 'total_points', 'total_steals', 'total_drebounds_pct',

Since RFE doesn't work with KNN, we will be using SelectKBest which produces the same process

In [9]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
results = []

for i in range(min_features, total_features):
    knn = KNeighborsClassifier()

    selector = SelectKBest(score_func=mutual_info_classif, k=i)
    selector.fit(x_train, y_train)


    selected_features = set(x_train.columns[selector.get_support()])
    selected_features.add("tmid")
    selected_features.add("year")
    selected_features = list(selected_features)

    knn.fit(x_train[selected_features], y_train)

    accuracy = knn.score(x_test[selected_features], y_test)

    results.append((selected_features, accuracy))

results = sorted(results, key=lambda x: x[1], reverse=True)

classifiers_features["K-Nearest Neighbors"] = results[0][0]

# Print the best 3 results
for features, accuracy in results[:3]:
    print("Selected Features:", features)
    print("Accuracy:" + str(accuracy) + '\n')

Selected Features: ['coach_po_wr', 'player_awards', 'coach_awards', 'team_players_rating', 'year', 'tmid']
Accuracy:0.6923076923076923

Selected Features: ['playoff_rank', 'total_assists', 'tmid', 'total_blocks', 'coach_reg_wr', 'team_playoffs_count', 'coach_po_wr', 'rank', 'total_points', 'total_steals', 'team_players_rating', 'total_gs', 'year', 'total_minutes']
Accuracy:0.6923076923076923

Selected Features: ['playoff_rank', 'total_assists', 'tmid', 'total_blocks', 'coach_reg_wr', 'player_awards', 'coach_awards', 'total_points', 'total_steals', 'total_dq', 'team_players_rating', 'team_rating', 'total_gs', 'year', 'total_minutes']
Accuracy:0.6923076923076923



In [10]:
accuracy_scores = []


for test_year in range(3, 11):  # Testing from year 2 to 10
    x_train = merged_data[merged_data['year'].between(1, test_year - 1)].drop('playoff', axis=1)
    y_train = merged_data[merged_data['year'].between(1, test_year - 1)]['playoff']
    
    x_test = merged_data[merged_data['year'] == test_year].drop('playoff', axis=1)
    y_test = merged_data[merged_data['year'] == test_year]['playoff']
    
    # Assuming you have the best model and features from the previous steps
    best_model = best_model_info[best_model_name]['model']
    best_features = best_model_info[best_model_name]['features']

    # Training the model
    best_model.fit(x_train[best_features], y_train)
    
    # Testing the model
    predictions = best_model.predict(x_test[best_features])
    
    # Calculating accuracy
    accuracy = accuracy_score(y_test, predictions)
    accuracy_scores.append(accuracy)
    print(f"Accuracy for testing year {test_year}: {accuracy}")

# Calculating average accuracy
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(f"\nAverage Accuracy across all years: {average_accuracy}")

Accuracy for testing year 3: 0.6875
Accuracy for testing year 4: 0.6428571428571429
Accuracy for testing year 5: 0.3076923076923077
Accuracy for testing year 6: 0.6153846153846154
Accuracy for testing year 7: 0.7857142857142857
Accuracy for testing year 8: 0.6153846153846154
Accuracy for testing year 9: 0.5
Accuracy for testing year 10: 0.8461538461538461

Average Accuracy across all years: 0.6250858516483516


In [11]:

x_train = merged_data[merged_data['year'].between(1, 10)].drop('playoff', axis=1)
y_train = merged_data[merged_data['year'].between(1, 10)]['playoff']

x_test = merged_data[merged_data['year'] == 11].drop('playoff', axis=1)
y_test = merged_data[merged_data['year'] == 11]['playoff']


best_model.fit(x_train[best_features], y_train)
predictions = best_model.predict(x_test[best_features])


print(merged_data[merged_data['year'] == 11])

y_t = [1,0,0,1,1,0,1,1,1,1,0,1]

print(accuracy_score(y_t, predictions))




     year  tmid  player_awards  total_minutes  total_points  total_assists   
126    11     0              3         6267.4        2382.0          507.2  \
127    11     2              1         3225.8        1166.6          208.6   
128    11     4              1         3374.4        1311.6          269.4   
129    11     7              6         5906.0        2501.6          446.2   
130    11     8              6         5208.8        1775.4          492.2   
131    11    10              3         4489.4        1867.8          409.8   
132    11    11              2         4154.4        1900.0          360.4   
133    11    13              6         4853.0        2207.8          428.0   
134    11    16              2         4497.8        1918.8          405.8   
135    11    17              8         6151.2        2408.2          487.8   
136    11    18             10         2455.8         896.2          185.6   
137    11    20              2         4056.6        1500.6     

#### GridSearch
Now that we know the best features for each model, we will use gridsearch to fine tune its parameters.

In [12]:
#best_params = fs.grid_search(classifiers_features,x_train,x_test,y_train,y_test)
#print(best_params)

# Random Forest -> {'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 50}

# Logistic -> {'C': 100, 'fit_intercept': False, 'penalty': 'l1', 'solver': 'liblinear'}

# SVM -> {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}

# Gradient -> {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 50}

# KNN ->  {'n_neighbors': 10, 'p': 2, 'weights': 'uniform'}

import time

model_params = {'Random Forest': {'random_state':42, 'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 50}, 'Logistic Regression': {'random_state':42, 'C': 100, 'fit_intercept': False, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 10000}, 'Support Vector Machine': {'random_state':42, 'C': 10, 'gamma': 'scale', 'kernel': 'linear'}, 'Gradient Boosting': {'random_state':42,'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 50}, 'K-Nearest Neighbors': {'n_neighbors': 10, 'p': 2, 'weights': 'uniform'}}

final_classifiers = {
    'Random Forest': RandomForestClassifier(random_state =42),
    'Logistic Regression': LogisticRegression(random_state =42),
    'Support Vector Machine': SVC(random_state =42),
    'Gradient Boosting': GradientBoostingClassifier(random_state =42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
}

for model_name, model in final_classifiers.items():
    start = time.time()
    model.fit(x_train[classifiers_features[model_name]], y_train)
    y_pred = model.predict(x_test[classifiers_features[model_name]])
    
    accuracy = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    end = time.time()
    print(f'Accuracy for {model_name}: {accuracy}')
    print(f'AUC for {model_name}: {auc}')
    print(end-start)


C:\Users\ricar\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\multiclass.py:380: RuntimeWarning: invalid value encountered in cast
  if xp.any(data != data.astype(int)):


ValueError: Input y_true contains NaN.